# Intro

In this notebook you will see how I cleaned data from this [dataset](https://www.kaggle.com/rikdifos/credit-card-approval-prediction) on credit cards to create a clean, easier to use version. You can find the resulting dataset [here](https://www.kaggle.com/samuelcortinhas/credit-card-classification-clean-data).

# Libraries

In [1]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from imblearn.over_sampling import SMOTE
import itertools

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import mutual_info_classif

from sklearn.ensemble import RandomForestClassifier

# Data

In [2]:
# Save to df
applications = pd.read_csv("./data/application_record.csv", encoding = 'utf-8')
record = pd.read_csv("./data/credit_record.csv", encoding = 'utf-8')

# Shape and preview
print('Application record df shape:', applications.shape)
applications.head()

Application record df shape: (438557, 18)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
# Shape and preview
print('Credit record df shape:', record.shape)
record.head()

Credit record df shape: (1048575, 3)


,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


**Missing values**

In [4]:
applications.isna().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [5]:
record.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

**Duplicates**

In [6]:
print(f'Duplicates in applications df: {applications.iloc[:,1:].duplicated().sum()}, ({np.round(100*applications.iloc[:,1:].duplicated().sum()/len(applications),1)}%)')
print('')
print(f'Duplicates in record df: {record.duplicated().sum()}, ({np.round(100*record.duplicated().sum()/len(record),1)}%)')

Duplicates in applications df: 348472, (79.5%)

Duplicates in record df: 0, (0.0%)


**Cardinality of features**

There is 1 constant feature, 6 binary features, 3 continuous features and 7 non-binary categorical features.

In [7]:
applications.nunique()

ID                     438510
CODE_GENDER                 2
FLAG_OWN_CAR                2
FLAG_OWN_REALTY             2
CNT_CHILDREN               12
AMT_INCOME_TOTAL          866
NAME_INCOME_TYPE            5
NAME_EDUCATION_TYPE         5
NAME_FAMILY_STATUS          5
NAME_HOUSING_TYPE           6
DAYS_BIRTH              16379
DAYS_EMPLOYED            9406
FLAG_MOBIL                  1
FLAG_WORK_PHONE             2
FLAG_PHONE                  2
FLAG_EMAIL                  2
OCCUPATION_TYPE            18
CNT_FAM_MEMBERS            13
dtype: int64

# Data Cleaning

**Drop duplicates**

In [8]:
# Drop duplicates
applications=applications.drop_duplicates(subset=applications.columns[1:], keep='first')

**Fill missing values**

In [9]:
# Fill missing values
applications['OCCUPATION_TYPE'].fillna(value='Other', inplace=True)

**Drop constant feature**

This won't be useful to our models.

In [10]:
# Drop constant feature
applications.drop('FLAG_MOBIL', axis=1, inplace=True)

**Create target variable**

Because the credit records are incomplete (the ID ranges in both dfs don't match) we have to create our own target. We do this by identifying 'low risk' and 'high risk' credit users. We say a user is 'high risk' if during any month they are late on payments by 30 days or more, and 'low risk' otherwise. This is the target we will try to predict later.

In [11]:
# Construct target
record['target']=record['STATUS']
record['target'].replace('X', 0, inplace=True)
record['target'].replace('C', 0, inplace=True)
record['target']=record['target'].astype(int)
record.loc[record['target']>=1,'target']=1

# target=1 (high risk) iff there is at least one month where user is late on payments by 30 days or more
target_df=pd.DataFrame(record.groupby(['ID'])['target'].agg(max)).reset_index()

# Merge dataframes
new_df=pd.merge(applications, target_df, how='inner', on=['ID'])

C:\TempFolder\ipykernel_11500\182635032.py:9: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  target_df=pd.DataFrame(record.groupby(['ID'])['target'].agg(max)).reset_index()


**Account length**

It makes sense that the number of months the account has been open for correlates with the riskiness of the customer (since there are more opportunity to miss payments).

In [12]:
# Extract how many months account has been open for
start_df=pd.DataFrame(record.groupby(['ID'])['MONTHS_BALANCE'].agg(min)).reset_index()

# Rename column
start_df.rename(columns={'MONTHS_BALANCE':'ACCOUNT_LENGTH'}, inplace=True)

# Make entries positive
start_df['ACCOUNT_LENGTH']=-start_df['ACCOUNT_LENGTH']

# Merge dataframes on ID
new_df=pd.merge(new_df, start_df, how='inner', on=['ID'])

C:\TempFolder\ipykernel_11500\1593942993.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'min' instead.
  start_df=pd.DataFrame(record.groupby(['ID'])['MONTHS_BALANCE'].agg(min)).reset_index()


**Continuous features**

In [13]:
# Create age feature
new_df['AGE_YEARS']=-new_df['DAYS_BIRTH']/365.2425
new_df.drop('DAYS_BIRTH', axis=1, inplace=True)

# Create unemployed indicator
new_df['UNEMPLOYED']=0
new_df.loc[-new_df['DAYS_EMPLOYED']<0,'UNEMPLOYED']=1

# Create years employed feature
new_df['YEARS_EMPLOYED']=-new_df['DAYS_EMPLOYED']/365.2425
new_df.loc[new_df['YEARS_EMPLOYED']<0,'YEARS_EMPLOYED']=0
new_df.drop('DAYS_EMPLOYED', axis=1, inplace=True)

**Encoding categorical features**

In [14]:
# Encode binary features
new_df["CODE_GENDER"] =  new_df["CODE_GENDER"].replace(['F','M'],[0,1])
new_df["FLAG_OWN_CAR"] = new_df["FLAG_OWN_CAR"].replace(["Y","N"],[1,0])
new_df["FLAG_OWN_REALTY"] = new_df["FLAG_OWN_REALTY"].replace(["Y","N"],[1,0])

**Rename columns**

In [15]:
new_df = new_df.rename(columns={'CODE_GENDER': 'Gender', 'FLAG_OWN_CAR': 'Own_car', 
                               'FLAG_OWN_REALTY':'Own_property','CNT_CHILDREN':'Num_children',
                               'AMT_INCOME_TOTAL':'Total_income','NAME_INCOME_TYPE':'Income_type',
                               'NAME_EDUCATION_TYPE':'Education_type','NAME_FAMILY_STATUS':'Family_status',
                               'NAME_HOUSING_TYPE':'Housing_type','FLAG_WORK_PHONE':'Work_phone',
                               'FLAG_PHONE':'Phone','FLAG_EMAIL':'Email','OCCUPATION_TYPE':'Occupation_type',
                               'CNT_FAM_MEMBERS':'Num_family','target':'Target','ACCOUNT_LENGTH':'Account_length',
                                'AGE_YEARS':'Age','UNEMPLOYED':'Unemployed','YEARS_EMPLOYED':'Years_employed'})

**Data type**

In [16]:
new_df['Num_family']=new_df['Num_family'].astype(int)

**Re-order columns**

In [17]:
new_df.columns

Index(['ID', 'Gender', 'Own_car', 'Own_property', 'Num_children',
       'Total_income', 'Income_type', 'Education_type', 'Family_status',
       'Housing_type', 'Work_phone', 'Phone', 'Email', 'Occupation_type',
       'Num_family', 'Target', 'Account_length', 'Age', 'Unemployed',
       'Years_employed'],
      dtype='object')

In [18]:
new_df=new_df[['ID', 'Gender', 'Own_car', 'Own_property', 'Work_phone',
               'Phone', 'Email', 'Unemployed', 'Num_children', 'Num_family', 
               'Account_length', 'Total_income', 'Age', 'Years_employed',  
               'Income_type', 'Education_type', 'Family_status',
               'Housing_type', 'Occupation_type','Target']]

# Output

In [19]:
# Shape and preview
print('New df shape:', new_df.shape)
new_df.head()

New df shape: (9709, 20)


,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,Working,Higher education,Civil marriage,Rented apartment,Other,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,Working,Secondary / secondary special,Married,House / apartment,Security staff,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,Pensioner,Higher education,Separated,House / apartment,Other,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,Working,Higher education,Married,House / apartment,Accountants,0


In [20]:
# Data types
new_df.dtypes

ID                   int64
Gender               int64
Own_car              int64
Own_property         int64
Work_phone           int64
Phone                int64
Email                int64
Unemployed           int64
Num_children         int64
Num_family           int32
Account_length       int64
Total_income       float64
Age                float64
Years_employed     float64
Income_type         object
Education_type      object
Family_status       object
Housing_type        object
Occupation_type     object
Target               int32
dtype: object

# LabelEncoding 

In [21]:
# Education_type 변수 label encoding
label_encoder = LabelEncoder()
new_df['Education_type'] = label_encoder.fit_transform(new_df['Education_type'])

# label과 실제 값의 매핑 확인
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Academic degree': 0, 'Higher education': 1, 'Incomplete higher': 2, 'Lower secondary': 3, 'Secondary / secondary special': 4}


In [22]:
# Education_type 변수 label encoding
label_encoder = LabelEncoder()
new_df['Income_type'] = label_encoder.fit_transform(new_df['Income_type'])

# label과 실제 값의 매핑 확인
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Commercial associate': 0, 'Pensioner': 1, 'State servant': 2, 'Student': 3, 'Working': 4}


In [23]:
# Education_type 변수 label encoding
label_encoder = LabelEncoder()
new_df['Family_status'] = label_encoder.fit_transform(new_df['Family_status'])

# label과 실제 값의 매핑 확인
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Civil marriage': 0, 'Married': 1, 'Separated': 2, 'Single / not married': 3, 'Widow': 4}


In [24]:
# Education_type 변수 label encoding
label_encoder = LabelEncoder()
new_df['Housing_type'] = label_encoder.fit_transform(new_df['Housing_type'])

# label과 실제 값의 매핑 확인
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Co-op apartment': 0, 'House / apartment': 1, 'Municipal apartment': 2, 'Office apartment': 3, 'Rented apartment': 4, 'With parents': 5}


In [25]:
# Education_type 변수 label encoding
label_encoder = LabelEncoder()
new_df['Occupation_type'] = label_encoder.fit_transform(new_df['Occupation_type'])

# label과 실제 값의 매핑 확인
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'Accountants': 0, 'Cleaning staff': 1, 'Cooking staff': 2, 'Core staff': 3, 'Drivers': 4, 'HR staff': 5, 'High skill tech staff': 6, 'IT staff': 7, 'Laborers': 8, 'Low-skill Laborers': 9, 'Managers': 10, 'Medicine staff': 11, 'Other': 12, 'Private service staff': 13, 'Realty agents': 14, 'Sales staff': 15, 'Secretaries': 16, 'Security staff': 17, 'Waiters/barmen staff': 18}


In [26]:
new_df

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500.0,32.868574,12.435574,4,1,0,4,12,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500.0,58.793815,3.104787,4,4,1,1,17,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000.0,52.321403,8.353354,0,4,3,1,15,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500.0,61.504343,0.000000,1,1,2,1,12,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000.0,46.193967,2.105450,4,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,5148694,0,0,0,0,0,0,0,0,2,20,180000.0,56.400884,0.542106,1,4,0,2,8,1
9705,5149055,0,0,1,1,1,0,0,0,2,19,112500.0,43.360233,7.375921,0,4,1,1,12,1
9706,5149729,1,1,1,0,0,0,0,0,2,21,90000.0,52.296762,4.711938,4,4,1,1,12,1
9707,5149838,0,0,1,0,1,1,0,0,2,32,157500.0,33.914454,3.627727,1,1,1,1,11,1


In [27]:
# float 타입 변수를 정수형으로 변환
float_cols = new_df.select_dtypes(include=['float']).columns
for col in float_cols:
    new_df[col] = new_df[col].astype(int)
    
new_df

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500,32,12,4,1,0,4,12,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500,58,3,4,4,1,1,17,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000,52,8,0,4,3,1,15,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500,61,0,1,1,2,1,12,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000,46,2,4,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,5148694,0,0,0,0,0,0,0,0,2,20,180000,56,0,1,4,0,2,8,1
9705,5149055,0,0,1,1,1,0,0,0,2,19,112500,43,7,0,4,1,1,12,1
9706,5149729,1,1,1,0,0,0,0,0,2,21,90000,52,4,4,4,1,1,12,1
9707,5149838,0,0,1,0,1,1,0,0,2,32,157500,33,3,1,1,1,1,11,1


In [28]:
# # Save to csv
new_df.to_csv('clean_data.csv',index=False)

In [29]:
new_df

,ID,Gender,Own_car,Own_property,Work_phone,Phone,Email,Unemployed,Num_children,Num_family,Account_length,Total_income,Age,Years_employed,Income_type,Education_type,Family_status,Housing_type,Occupation_type,Target
0,5008804,1,1,1,1,0,0,0,0,2,15,427500,32,12,4,1,0,4,12,1
1,5008806,1,1,1,0,0,0,0,0,2,29,112500,58,3,4,4,1,1,17,0
2,5008808,0,0,1,0,1,1,0,0,1,4,270000,52,8,0,4,3,1,15,0
3,5008812,0,0,1,0,0,0,1,0,1,20,283500,61,0,1,1,2,1,12,0
4,5008815,1,1,1,1,1,1,0,0,2,5,270000,46,2,4,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,5148694,0,0,0,0,0,0,0,0,2,20,180000,56,0,1,4,0,2,8,1
9705,5149055,0,0,1,1,1,0,0,0,2,19,112500,43,7,0,4,1,1,12,1
9706,5149729,1,1,1,0,0,0,0,0,2,21,90000,52,4,4,4,1,1,12,1
9707,5149838,0,0,1,0,1,1,0,0,2,32,157500,33,3,1,1,1,1,11,1


In [30]:
from sklearn.model_selection import train_test_split

# 종속변수(Target)와 독립변수 분리
X = new_df.drop('Target', axis=1)
y = new_df['Target']

# train, test 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
X_train.to_csv("X_train.csv", index= False)

In [36]:
y_train.to_csv("y_train.csv", index= False)

In [31]:
X_test.to_csv("X_test.csv", index= False)

In [32]:
y_test.to_csv("y_test.csv", index= False)

In [31]:
# # X_train, y_train 합치기
# train_data = pd.concat([X_train, y_train], axis=1)

# # X_test, y_test 합치기
# test_data = pd.concat([X_test, y_test], axis=1)

In [33]:
# # train_data 저장
# train_data.to_csv('train_data.csv', index=False)

# # test_data 저장
# test_data.to_csv('test_data.csv', index=False)